In [71]:
import numpy as np 
import pandas as pd
import os
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import cv2
import albumentations as A
from PIL import Image
from albumentations.pytorch import ToTensorV2

In [72]:
fol = np.sort(os.listdir('/kaggle/input/dermnet/train'))
fol
dic ={}
for i,x in enumerate(fol):
    dic[x]=i
dic
#ID La

{'Acne and Rosacea Photos': 0,
 'Actinic Keratosis Basal Cell Carcinoma and other Malignant Lesions': 1,
 'Atopic Dermatitis Photos': 2,
 'Bullous Disease Photos': 3,
 'Cellulitis Impetigo and other Bacterial Infections': 4,
 'Eczema Photos': 5,
 'Exanthems and Drug Eruptions': 6,
 'Hair Loss Photos Alopecia and other Hair Diseases': 7,
 'Herpes HPV and other STDs Photos': 8,
 'Light Diseases and Disorders of Pigmentation': 9,
 'Lupus and other Connective Tissue diseases': 10,
 'Melanoma Skin Cancer Nevi and Moles': 11,
 'Nail Fungus and other Nail Disease': 12,
 'Poison Ivy Photos and other Contact Dermatitis': 13,
 'Psoriasis pictures Lichen Planus and related diseases': 14,
 'Scabies Lyme Disease and other Infestations and Bites': 15,
 'Seborrheic Keratoses and other Benign Tumors': 16,
 'Systemic Disease': 17,
 'Tinea Ringworm Candidiasis and other Fungal Infections': 18,
 'Urticaria Hives': 19,
 'Vascular Tumors': 20,
 'Vasculitis Photos': 21,
 'Warts Molluscum and other Viral Inf

In [73]:
dftrain = pd.DataFrame(columns=['id','label','y'])

for folder in fol:
    ls = os.listdir('/kaggle/input/dermnet/train/'+folder)
    for x in ls:
        #print(folder)
        dftrain.loc[len(dftrain)] = {'id':x,'label':folder,'y':dic[folder]}
dftrain

,id,label,y
0,milia-11.jpg,Acne and Rosacea Photos,0
1,acne-cystic-102.jpg,Acne and Rosacea Photos,0
2,acne-infantile-2.jpg,Acne and Rosacea Photos,0
3,acne-cystic-146.jpg,Acne and Rosacea Photos,0
4,acne-cystic-6.jpg,Acne and Rosacea Photos,0
...,...,...,...
15552,warts-common-55.jpg,Warts Molluscum and other Viral Infections,22
15553,warts-40.jpg,Warts Molluscum and other Viral Infections,22
15554,warts-cryotherapy-17.jpg,Warts Molluscum and other Viral Infections,22
15555,molluscum-contagiosum-124.jpg,Warts Molluscum and other Viral Infections,22


In [74]:
import cv2 as cv

In [75]:
dftest = pd.DataFrame(columns=['id','label','y'])

for folder in fol:
    ls = os.listdir('/kaggle/input/dermnet/test/'+folder)
    for x in ls:
        #print(folder)
        dftest.loc[len(dftest)] = {'id':x,'label':folder,'y':dic[folder]}
dftest

,id,label,y
0,acne-cystic-6.jpg,Acne and Rosacea Photos,0
1,hidradenitis-suppurativa-43.jpg,Acne and Rosacea Photos,0
2,acne-open-comedo-67.jpg,Acne and Rosacea Photos,0
3,perioral-dermatitis-10.jpg,Acne and Rosacea Photos,0
4,hyperhidrosis-2.jpg,Acne and Rosacea Photos,0
...,...,...,...
3997,warts-common-96.jpg,Warts Molluscum and other Viral Infections,22
3998,herpes-simplex-37.jpg,Warts Molluscum and other Viral Infections,22
3999,herpes-zoster-182.jpg,Warts Molluscum and other Viral Infections,22
4000,molluscum-contagiosum-155.jpg,Warts Molluscum and other Viral Infections,22


In [76]:

from sklearn.model_selection import train_test_split

# given:
# features: xs
# ground truth: ys

x_train, x_val, y_train, y_val = train_test_split(dftrain['id'], dftrain['y'],
                                                    test_size=0.33,
                                                    random_state=0,
                                                    stratify=dftrain['y'])

In [77]:
transform = A.Compose([
#         A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=15, p=0.5),
        A.CenterCrop(height=300, width=300),
                
        A.HorizontalFlip(p=0.3),
        A.VerticalFlip(p=0.3),
#         A.RGBShift(r_shift_limit=15, g_shift_limit=15, b_shift_limit=15, p=0.5),
        A.RandomBrightnessContrast(p=0.5),
        A.Normalize(mean=(0.485, 0.456, 0.406), std=(0.229, 0.224, 0.225)),
#         FiveCrop(405), # this is a list of PIL Images
#         Lambda(lambda crops: torch.stack([PILToTensor()(crop) for crop in crops])),
        ToTensorV2()
])

In [78]:
class Data(Dataset):
    def __init__(self,folder,train='True',test=True):
        self.train=train
        self.folder = folder
        self.lis = np.sort(os.listdir(folder))
        self.transform = transform
        self.test=test
    def gammaCorrection(self,src, gamma=2.5):
        cv2_image = np.transpose(src, (1, 2, 0))
        cv2_image = cv2.cvtColor(cv2_image, cv2.COLOR_BGR2RGB)
        invGamma = 1.0/2.5

        table = [((i / 255) ** invGamma) * 255 for i in range(256)]
        table = np.array(table, np.uint8)
        ret['x'] =cv.LUT(src, table) 
        return ret
    def __len__(self):
        if self.train==True:
            return x_train.shape[0]
        elif self.test==False:
            return x_val.shape[0]
        else:
            return dftest.shape[0]
    
    def __getitem__(self,idx):
        if self.train==True:
            file_path = '/kaggle/input/dermnet/train/'+fol[y_train.iloc[idx]]+'/'+x_train.iloc[idx]
            y=y_train.iloc[idx]
        elif self.train==False and self.test==False:
            file_path = '/kaggle/input/dermnet/train/'+fol[y_val.iloc[idx]]+'/'+x_val.iloc[idx]
            y = y_val.iloc[idx]
        else:
            file_path = '/kaggle/input/dermnet/test/'+dftest['label'].iloc[idx]+'/'+dftest['id'].iloc[idx]
            y = dftest['y'].iloc[idx]
        
#         else:
#             y = torch.tensor(1)
#             file_path = self.folder+'/'+self.lis[1]+'/'+np.sort(os.listdir(self.folder+'/'+self.lis[1]))[idx-len(os.listdir(self.folder+'/'+self.lis[0]))]
        
        img = np.array(Image.open(file_path))
        img = self.transform(image=img)
        #img = self.gammaCorrection(img)
        return {'x': img, 'y':y}

In [79]:
dts = Data('/kaggle/input/dermnet/train',train=True,test=False)
dval = Data('/kaggle/input/dermnet/train',train=False,test=False)
dtest=Data('/kaggle/input/dermnet/test',train=False,test=True)

In [80]:
import torchvision
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [81]:
model = torchvision.models.resnet18(pretrained=True)
for param in model.parameters():
     param.requires_grad = False
num_features = model.fc.in_features
model.fc = nn.Linear(in_features=num_features, out_features=23)
for param in model.fc.parameters():
    param.requires_grad=True
# model

/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/conda/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [82]:
train_loader = DataLoader(dataset=dts,batch_size=32,shuffle=True)
val_loader = DataLoader(dataset=dval,batch_size=32,shuffle=False)
testloader=DataLoader(dataset=dtest,batch_size=32,shuffle=False)

In [83]:
from sklearn.metrics import accuracy_score
from tqdm.notebook import tqdm
# import torchmetrics.cl
from torchmetrics import F1Score
from torchmetrics.classification import MulticlassF1Score

In [84]:
f1=MulticlassF1Score(num_classes=23).to(device)
import gc
torch.cuda.empty_cache()
gc.collect()

1952

In [85]:
import torch.nn.functional as F

class FocalLoss(nn.modules.loss._WeightedLoss):
    def __init__(self, weight=None, gamma=4,reduction='mean'):
        super(FocalLoss, self).__init__(weight,reduction=reduction)
        self.gamma = gamma
        self.weight = weight #weight parameter will act as the alpha parameter to balance class weights

    def forward(self, input, target):

        ce_loss = F.cross_entropy(input, target,reduction=self.reduction,weight=self.weight)
        pt = torch.exp(-ce_loss)
        focal_loss = ((1 - pt) ** self.gamma * ce_loss).mean()
        return focal_loss

In [86]:
loss_fn = FocalLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=1e-3)
def epoc(model,epochs=1):
    
    for epoch in range(epochs):
        model.train()
    #     acc=0
        loss_train = 0
        for data in tqdm(train_loader,total = len(train_loader)):
            torch.cuda.empty_cache()
            model = model.to(device)
            for k,v in data.items():
                if type(v)==dict:
                    data[k] = v['image'].to(device)
                else:
                    data[k]=v.to(device)
            x = data['x']
            optimizer.zero_grad()
            outs = model(x)
    #         print(data['y'])
            outs = nn.Softmax(dim=1)(outs)

    #       
            one_hot =torch.tensor( torch.nn.functional.one_hot(data['y'],23 ),dtype=float)
            #print(data['y'])
            loss = loss_fn(outs,one_hot)
            loss.backward()
            optimizer.step()
            outs=outs.argmax(axis=1)
            loss_train += loss.item()
            f11=f1(outs,data['y'])
            print(f11)
    #         acc=torch.sum(outs==data['y'])/128

    #         print(acc)
            for k,v in data.items():
                if type(v)==dict:
                    data[k] = v['image'].to('cpu')
                else:
                    data[k]=v.to('cpu')

    #     print(loss_train/len(train_loader))

    #     print(acc)

    #         l.extend(outs)
    #         len(l)
    #         l2.extend(data['y'])
    #         print(outs.shape,data['y'].shape)
    # print(len(l))    

In [ ]:
epoc(model,2)

  0%|          | 0/326 [00:00<?, ?it/s]

/tmp/ipykernel_32/4040144701.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  one_hot =torch.tensor( torch.nn.functional.one_hot(data['y'],23 ),dtype=float)


tensor(0.0299)
tensor(0.0104)
tensor(0.0078)
tensor(0.0211)
tensor(0.0038)
tensor(0.0132)
tensor(0.0074)
tensor(0.0169)
tensor(0.0074)
tensor(0.0114)
tensor(0.0065)
tensor(0.)
tensor(0.0114)
tensor(0.0276)
tensor(0.0208)
tensor(0.0069)
tensor(0.0246)
tensor(0.0030)
tensor(0.0051)
tensor(0.0065)
tensor(0.0118)
tensor(0.0090)
tensor(0.0084)
tensor(0.0110)
tensor(0.0333)
tensor(0.0472)
tensor(0.0103)
tensor(0.0592)
tensor(0.0539)
tensor(0.0110)
tensor(0.0101)
tensor(0.0143)
tensor(0.0132)
tensor(0.0159)
tensor(0.0074)
tensor(0.0084)
tensor(0.0069)
tensor(0.0122)
tensor(0.0047)
tensor(0.0143)
tensor(0.0417)
tensor(0.0822)
tensor(0.0422)
tensor(0.0154)
tensor(0.)
tensor(0.0059)
tensor(0.0278)
tensor(0.0235)
tensor(0.0699)
tensor(0.0625)
tensor(0.0196)
tensor(0.0298)
tensor(0.0631)
tensor(0.0480)
tensor(0.0586)
tensor(0.0611)
tensor(0.0681)
tensor(0.0562)
tensor(0.0492)
tensor(0.0316)
tensor(0.0410)
tensor(0.0440)
tensor(0.0472)
tensor(0.0179)
tensor(0.0301)
tensor(0.0302)
tensor(0.0292)
ten

In [ ]:
model.eval()
l2=torch.empty((0,1), dtype=torch.float32,device=device)
with torch.no_grad():
    for data in tqdm(val_loader,total = len(val_loader)):
        torch.cuda.empty_cache()
        model = model.to(device)
        for k,v in data.items():
            if type(v)==dict:
                data[k] = v['image'].to(device)
            else:
                data[k]=v.to(device)
        x = data['x']
        outs = model(x)

        outs = nn.Softmax(dim=1)(outs)
        outs=outs.argmax(axis=1)
        f11=f1(outs,data['y'])
        outs=outs.view((-1,1))
        l=torch.cat((l,outs),0)
        print(f11)

In [ ]:
with torch.no_grad():
    l2=l2
    print(dfval['y'].shape)
    print(l2.shape)
    arr = dfval['y'].to_numpy().reshape(-1,1)
    f11=f1(l2,torch.tensor(arr,device=device))
    print(f11)

In [ ]:
l = torch.empty((0,1), dtype=torch.float32,device=device)
with torch.no_grad():
    for data in tqdm(testloader,total = len(testloader)):
        torch.cuda.empty_cache()
        model = model.to(device)
        for k,v in data.items():
            if type(v)==dict:
                data[k] = v['image'].to(device)
            else:-
                data[k]=v.to(device)
        x = data['x']
        outs = model(x)

        outs = nn.Softmax(dim=1)(outs)
        outs=outs.argmax(axis=1)
        
#         
        f11=f1(outs,data['y'])
        print(f11)
        outs=outs.view((-1,1))
        l=torch.cat((l,outs),0)
        print(outs.shape)

In [ ]:
with torch.no_grad():
    l=l
    print(dftest['y'].shape)
    print(l.shape)
    arr = dftest['y'].to_numpy().reshape(-1,1)
    f11=f1(l,torch.tensor(arr,device=device))
    print(f11)

In [ ]:
%cd /kaggle/working/

In [ ]:
import pickle 
pickle_out = open("classifier.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close

In [ ]:
%cd ..

In [ ]:
pickle_out = open("classifier.pkl","wb")
pickle.dump(model,pickle_out)
pickle_out.close

In [ ]:
import os
import subprocess
from IPython.display import FileLink, display

def download_file(path, download_file_name):
    os.chdir('/kaggle/working/')
    zip_name = f"/kaggle/working/{download_file_name}.zip"
    command = f"zip {zip_name} {path} -r"
    result = subprocess.run(command, shell=True, capture_output=True, text=True)
    if result.returncode != 0:
        print("Unable to run zip command!")
        print(result.stderr)
        return
    display(FileLink(f'{download_file_name}.zip'))

In [ ]:
download_file('/kaggle/working/classifier.pkl','classifierpkl')